# Net Position Analysis: From Net-to-Robot to Robot-to-Net

This notebook analyzes the relationship between:
1. **Net position from robot's perspective** (sonar-detected distance and angle)
2. **Robot position relative to the net** (navigation-based positioning)

The analysis transforms between these two coordinate systems using CSV data from net detection results and navigation data.

## 1. Import Libraries and Utils

Import necessary libraries and our custom net position analysis utilities.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

# Import our custom net position analysis utilities
from utils.net_position_analysis import (
    load_net_analysis_results,
    load_navigation_data,
    calculate_robot_to_net_position,
    analyze_net_position_consistency,
    create_net_position_visualization,
    run_net_position_analysis
)

# Import configuration
from utils.sonar_config import EXPORTS_DIR_DEFAULT

print("Libraries and utilities imported successfully!")

Libraries and utilities imported successfully!


## 2. Load Net Results from CSV

Load the net analysis results that contain sonar-detected distances and angles to the net from the robot's perspective.

In [2]:
# Configuration - set your target bag name here
TARGET_BAG = "2024-08-20_14-31-29"  # Replace with your actual bag name
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

print(f"Target Bag: {TARGET_BAG}")
print(f"Exports Folder: {EXPORTS_FOLDER}")

# Load net analysis results (sonar detections from robot perspective)
try:
    net_df = load_net_analysis_results(TARGET_BAG, EXPORTS_FOLDER)
    print(f"✓ Loaded {len(net_df)} net analysis frames")
    print(f"Columns: {list(net_df.columns)}")
    print(f"Detection success rate: {net_df['detection_success'].mean()*100:.1f}%")
except FileNotFoundError as e:
    print(f"✗ Error loading net results: {e}")
    print("Make sure you have run the sonar analysis and saved results to CSV first.")

Target Bag: 2024-08-20_14-31-29
Exports Folder: /Volumes/LaCie/SOLAQUA/exports
Loading net analysis results: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones_analysis_results.csv
✓ Loaded 1059 net analysis frames
Columns: ['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees', 'distance_meters', 'detection_success', 'tracking_status', 'area', 'aspect_ratio', 'solidity', 'extent', 'ellipse_elongation', 'straightness', 'rect', 'centroid_x', 'centroid_y']
Detection success rate: 100.0%


In [3]:
# Load navigation data (robot position and orientation)
try:
    nav_df = load_navigation_data(TARGET_BAG, EXPORTS_FOLDER)
    print(f"✓ Loaded {len(nav_df)} navigation frames")
    print(f"Navigation columns: {list(nav_df.columns)}")

    # Show sample of navigation data
    print("\nSample navigation data:")
    print(nav_df.head())

except FileNotFoundError as e:
    print(f"✗ Error loading navigation data: {e}")

Loading navigation data: /Volumes/LaCie/SOLAQUA/exports/by_bag/navigation_plane_approximation__2024-08-20_14-31-29_data.csv
✓ Loaded 559 navigation frames
Navigation columns: ['t', 't_header', 't_bag', 't_src', 'bag', 'bag_file', 'topic', 'NormalDVL', 'Altitude', 'NetDistance', 'NetHeading', 'NetPitch', 'NetLock', 'NetVelocity_u', 'NetVelocity_v', 'NetVelocity_w', '__msgtype__', 't0', 't_rel', 'ts_utc', 'ts_oslo', 'timestamp']

Sample navigation data:
              t      t_header         t_bag   t_src                       bag  \
0  1.724157e+09  1.724157e+09  1.724157e+09  header  2024-08-20_14-31-29_data   
1  1.724157e+09  1.724157e+09  1.724157e+09  header  2024-08-20_14-31-29_data   
2  1.724157e+09  1.724157e+09  1.724157e+09  header  2024-08-20_14-31-29_data   
3  1.724157e+09  1.724157e+09  1.724157e+09  header  2024-08-20_14-31-29_data   
4  1.724157e+09  1.724157e+09  1.724157e+09  header  2024-08-20_14-31-29_data   

                       bag_file                          

## 3. Calculate Robot Position from Net Position

Transform from net-relative coordinates (sonar distance/angle) to world coordinates using navigation data.

In [4]:
# Calculate robot-to-net positions (transform sonar detections to world coordinates)
print("Calculating robot-to-net positions...")
combined_df = calculate_robot_to_net_position(net_df, nav_df)

print(f"✓ Generated {len(combined_df)} position calculations")
print(f"Successful detections: {combined_df['detection_success'].sum()}")

# Show sample of combined data
print("\nSample combined data:")
print(combined_df.head())

# Summary statistics
print("\nPosition Summary:")
print(f"Mean sonar distance: {combined_df['sonar_distance_m'].mean():.2f} m")
print(f"Mean sonar angle: {combined_df['sonar_angle_deg'].mean():.2f}°")
print(f"Net position range - X: {combined_df['net_x_world'].min():.2f} to {combined_df['net_x_world'].max():.2f} m")
print(f"Net position range - Y: {combined_df['net_y_world'].min():.2f} to {combined_df['net_y_world'].max():.2f} m")

Calculating robot-to-net positions...
✓ Generated 1059 position calculations
Successful detections: 1059

Sample combined data:
                            timestamp  frame_index  detection_success  \
0 2024-08-20 12:31:31.621807575+00:00            1               True   
1 2024-08-20 12:31:31.681648731+00:00            2               True   
2 2024-08-20 12:31:31.742511034+00:00            3               True   
3 2024-08-20 12:31:31.818883181+00:00            4               True   
4 2024-08-20 12:31:31.874155045+00:00            5               True   

   sonar_distance_m  sonar_angle_deg  robot_x  robot_y  robot_heading  \
0          2.387373       209.268875        0        0              0   
1          2.390208       209.701591        0        0              0   
2          2.355667       208.376633        0        0              0   
3          2.346505       208.530899        0        0              0   
4          2.349601       209.089966        0        0              

## 4. Analyze Position Consistency

Analyze the consistency of net position estimates and calculate statistics.

In [5]:
# Analyze consistency of net position estimates
consistency_stats = analyze_net_position_consistency(combined_df)

print("=== POSITION CONSISTENCY ANALYSIS ===")
for key, value in consistency_stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Filter to successful detections for detailed analysis
success_df = combined_df[combined_df['detection_success']].copy()
print(f"\nDetailed analysis of {len(success_df)} successful detections:")

if len(success_df) > 0:
    # Calculate position variability
    position_variability = np.sqrt(
        (success_df['net_x_world'] - consistency_stats['net_centroid_x'])**2 +
        (success_df['net_y_world'] - consistency_stats['net_centroid_y'])**2
    )

    print(f"95th percentile position error: {np.percentile(position_variability, 95):.3f} m")
    print(f"Maximum position error: {position_variability.max():.3f} m")

=== POSITION CONSISTENCY ANALYSIS ===
total_frames: 1059
successful_detections: 1059
detection_rate: 100.000
net_centroid_x: -1.926
net_centroid_y: -0.368
position_std_m: 0.393
position_95th_percentile_m: 1.246
sonar_distance_mean: 2.037
sonar_distance_std: 0.327
sonar_angle_std: 15.259

Detailed analysis of 1059 successful detections:
95th percentile position error: 1.246 m
Maximum position error: 2.147 m


## 5. Visualize Net Position Over Time

Create focused plots showing how the estimated net position changes over time in X and Y coordinates separately.

In [6]:
# Create focused visualization of net position over time
fig = create_net_position_visualization(combined_df, consistency_stats)

# Display the figure
fig.show()

# Optional: Save the figure as HTML
# fig.write_html(f"{TARGET_BAG}_net_position_over_time.html")
# print(f"Visualization saved as {TARGET_BAG}_net_position_over_time.html")

print(f"Visualization shows {len(combined_df[combined_df['detection_success']])} successful detections")
print(f"Net position centroid: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m")

Visualization shows 1059 successful detections
Net position centroid: (-1.93, -0.37) m


## 6. Save Results to CSV

Save the complete analysis results to a CSV file for further processing or archiving.

In [7]:
# Save complete analysis results to CSV
from utils.sonar_config import EXPORTS_SUBDIRS

outputs_dir = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs', 'outputs')
outputs_dir.mkdir(parents=True, exist_ok=True)

results_filename = f"{TARGET_BAG}_net_position_analysis.csv"
results_path = outputs_dir / results_filename

combined_df.to_csv(results_path, index=False)
print(f"✓ Analysis results saved to: {results_path}")

# Also save consistency statistics as JSON
import json
stats_filename = f"{TARGET_BAG}_net_position_stats.json"
stats_path = outputs_dir / stats_filename

with open(stats_path, 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    json_stats = {k: (v.item() if hasattr(v, 'item') else v) for k, v in consistency_stats.items()}
    json.dump(json_stats, f, indent=2)

print(f"✓ Consistency statistics saved to: {stats_path}")

print(f"\n=== ANALYSIS COMPLETE ===")
print(f"Processed {len(combined_df)} frames with {consistency_stats['successful_detections']} successful detections")
print(f"Detection rate: {consistency_stats['detection_rate']:.1f}%")
print(f"Net position estimated at: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m")
print(f"Position consistency (STD): {consistency_stats['position_std_m']:.2f} m")

✓ Analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_analysis.csv
✓ Consistency statistics saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_stats.json

=== ANALYSIS COMPLETE ===
Processed 1059 frames with 1059 successful detections
Detection rate: 100.0%
Net position estimated at: (-1.93, -0.37) m
Position consistency (STD): 0.39 m


## Summary

This notebook analyzes net position estimates over time by transforming sonar detections from robot-centric coordinates to world coordinates.

### Key Results:
- **Detection Rate**: {consistency_stats['detection_rate']:.1f}%
- **Net Position Centroid**: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m
- **Position Consistency**: {consistency_stats['position_std_m']:.2f} m standard deviation

### Visualization:
- **Top Plot**: Net X position over time with centroid reference line
- **Bottom Plot**: Net Y position over time with centroid reference line

### Files Generated:
- `{TARGET_BAG}_net_position_analysis_complete.csv` - Complete analysis results
- `{TARGET_BAG}_net_position_stats.json` - Consistency statistics
- Interactive time-series plots

### Next Steps:
- Analyze position stability and detection reliability
- Compare with ground truth if available
- Tune sonar processing parameters for better consistency

In [8]:
# Save complete analysis results to CSV
output_file = EXPORTS_FOLDER / "outputs" / f"{TARGET_BAG}_net_position_analysis_complete.csv"

# Create outputs directory if it doesn't exist
output_file.parent.mkdir(parents=True, exist_ok=True)

# Save the combined dataframe with all analysis results
combined_df.to_csv(output_file, index=False)
print(f"✓ Complete analysis results saved to: {output_file}")

# Also save consistency statistics as a separate summary file
stats_file = EXPORTS_FOLDER / "outputs" / f"{TARGET_BAG}_net_position_stats.json"

import json
with open(stats_file, 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    json_stats = {k: float(v) if isinstance(v, (np.floating, np.integer)) else v
                  for k, v in consistency_stats.items()}
    json.dump(json_stats, f, indent=2)

print(f"✓ Analysis statistics saved to: {stats_file}")

print("\n=== ANALYSIS COMPLETE ===")
print(f"Processed {len(combined_df)} frames with {consistency_stats['detection_rate']:.1f}% detection rate")
print(f"Net estimated at world coordinates: ({consistency_stats['net_centroid_x']:.2f}, {consistency_stats['net_centroid_y']:.2f}) m")
print(f"Position consistency (STD): {consistency_stats['position_std_m']:.2f} m")

✓ Complete analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_analysis_complete.csv
✓ Analysis statistics saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_net_position_stats.json

=== ANALYSIS COMPLETE ===
Processed 1059 frames with 100.0% detection rate
Net estimated at world coordinates: (-1.93, -0.37) m
Position consistency (STD): 0.39 m


## Summary

This notebook successfully transformed between two coordinate systems:

1. **Net-relative coordinates** (sonar distance/angle from robot to net)
2. **World coordinates** (robot position relative to net in global reference frame)

### Key Results:
- **Detection Rate**: Percentage of frames with successful net detection
- **Net Centroid**: Most likely net position in world coordinates
- **Position Consistency**: How stable the net position estimates are over time
- **Time Series**: Separate X and Y position plots showing temporal evolution

### Files Created:
- `{TARGET_BAG}_net_position_analysis_complete.csv`: Full analysis results
- `{TARGET_BAG}_net_position_stats.json`: Summary statistics
- Interactive time-series visualization plots

### Next Steps:
- Validate coordinate transformations with ground truth data
- Analyze detection failures and improve sonar processing
- Use position estimates for navigation planning